In [1]:
%pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pickle
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix

import xmltodict
import json

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
train_text_xml = open("news_eval_train.xml", "r", encoding = "WINDOWS-1251")
train_text_xml = xmltodict.parse(train_text_xml.read())
train_json = json.loads(json.dumps(train_text_xml))

test_text_xml = open("news_eval_test.xml", "r", encoding = "WINDOWS-1251")
test_text_xml = xmltodict.parse(test_text_xml.read())
test_json = json.loads(json.dumps(test_text_xml))

In [3]:
#raw text
train_json["document"]["sentence"][1]["speech"]["#text"]

'"Меня отпустили. У Коли @nlyaskin забирают вещи и сажают. Хренникова забирает скорая, ему стало плохо в камере. Суд завтра в 15 Новая Площадь 8,стр. 3", - написала Чирикова в своем блоге в Twitter."В отношении Е.Чириковой составлен административный протокол по статье 19.3 КоАП города Москвы ("неповиновение законным требованиям сотрудникам полиции").'

In [4]:
#rating
train_json["document"]["sentence"][1]["evaluation"]

'-'

In [5]:
train_data = []

for elem in enumerate(train_json["document"]["sentence"]):
    # print(elem[1])
    raw_text = elem[1]["speech"]["#text"]
    rating = elem[1]["evaluation"]
    if rating in ["-", "0", "+"]:
        if rating == "-":
            rating = -1
        elif rating == "0":
            rating = 0
        elif rating == "+":
            rating = 1
    else:
        rating = None
    train_data.append([raw_text, rating])

In [6]:
test_data = []

for elem in enumerate(test_json["document"]["sentence"]):
    # print(elem[1])
    raw_text = elem[1]["speech"]["#text"]
    rating = elem[1]["evaluation"]
    if rating in ["-", "0", "+"]:
        if rating == "-":
            rating = -1
        elif rating == "0":
            rating = 0
        elif rating == "+":
            rating = 1
    else:
        rating = None
    test_data.append([raw_text, rating])

In [7]:
len(test_data)

5500

In [8]:
len(train_data)

4260

In [9]:
#убираем предложения без рейтинга
clear_test = []
clear_train = []

for elem in test_data:
    if elem[1] != None:
        clear_test.append(elem)
        
for elem in train_data:
    if elem[1] != None:
        clear_train.append(elem)

In [10]:
clear_test[1]

['"Посредством этих структур\nдесяткам тысяч избирателей предлагают деньги в обмен на паспортные\nданные и подписи за какого-либо кандидата", - сказал\nЧерненко.',
 -1]

In [11]:
clear_train[1][0]

'"Меня отпустили. У Коли @nlyaskin забирают вещи и сажают. Хренникова забирает скорая, ему стало плохо в камере. Суд завтра в 15 Новая Площадь 8,стр. 3", - написала Чирикова в своем блоге в Twitter."В отношении Е.Чириковой составлен административный протокол по статье 19.3 КоАП города Москвы ("неповиновение законным требованиям сотрудникам полиции").'

In [12]:
def filtered_words(tokens, stop_words):
    filtered_tokens = []
    for token in tokens:
        if token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens

In [13]:
#токенизация
import re
import pymorphy2, nltk
morph = pymorphy2.MorphAnalyzer(lang = 'ru')


nltk.download('stopwords')

all_words = []
stop_words = stopwords.words("russian")
marks = '''!()-[]{};?@#$%:'"\,./^&amp;*_—»«+=><°'''

for elem in clear_test:
    norm_sent = []
    text = elem[0].strip().lower()
    
    # удаление знаков препинания
    for x in text:  
        if x in marks:  
            text = text.replace(x, "")
            
    #удаление английских символов и цифр
    text = re.sub(r'[a-zA-Z0-9]', "", text)
    
    text = word_tokenize(text, language = "russian")
    
    #удаление стоп-слов
    text = filtered_words(text, stop_words)
    
    # нормализация
    for word in text:
        norm_sent.append(morph.parse(word)[0].normal_form)
    text = norm_sent
    
    elem[0] = text
    
    all_words.extend(text)

# print(all_words)
for elem in clear_train:
    norm_sent = []
    text = elem[0].strip().lower()
    
    # удаление знаков препинания
    for x in text:  
        if x in marks:  
            text = text.replace(x, "")
            
    #удаление английских символов и цифр
    text = re.sub(r'[a-zA-Z0-9]', "", text)
    
    text = word_tokenize(text, language = "russian")
    
    #удаление стоп-слов
    text = filtered_words(text, stop_words)
    
    # нормализация
    for word in text:
        norm_sent.append(morph.parse(word)[0].normal_form)
    text = norm_sent
    
    elem[0] = text
    
    all_words.extend(text)
    
    
all_words = sorted(list(set(all_words)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
all_words[10000:10010]

['мужество',
 'мужик',
 'мужской',
 'мужчие',
 'мужчина',
 'мужчинаодный',
 'музей',
 'музейзаповедник',
 'музейный',
 'музенидис']

In [15]:
len(all_words) #без нормализации было 44к

22379

In [16]:
clear_test[1]

[['посредством',
  'этот',
  'структура',
  'десятка',
  'тысяча',
  'избиратель',
  'предлагать',
  'деньга',
  'обмен',
  'паспортный',
  'дать',
  'подпись',
  'какоголибо',
  'кандидат',
  'сказать',
  'черненко'],
 -1]

In [17]:
clear_test = np.array(clear_test)
clear_train = np.array(clear_train)

C:\Users\user\AppData\Local\Temp/ipykernel_11788/861610473.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clear_test = np.array(clear_test)
C:\Users\user\AppData\Local\Temp/ipykernel_11788/861610473.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clear_train = np.array(clear_train)


In [22]:
main_clear_test = clear_test.copy()
main_clear_train = clear_train.copy()

In [23]:
main_clear_train[1]

array([list(['отпустить', 'коли', 'забирать', 'вещь', 'сажать', 'хренников', 'забирать', 'скорый', 'стать', 'плохо', 'камера', 'суд', 'завтра', 'новый', 'площадь', 'стр', 'написать', 'чириков', 'свой', 'блог', 'отношение', 'ечириков', 'составить', 'административный', 'протокол', 'статья', 'коап', 'город', 'москва', 'неповиновение', 'законный', 'требование', 'сотрудник', 'полиция']),
       -1], dtype=object)

In [24]:
main_clear_train is clear_train

False

***Булевы вектора***

In [25]:
#создание булевых векторов

for elem in clear_train:
    bool_vec = [0] * len(all_words)
    
    for word in elem[0]:
        bool_vec[all_words.index(word)] = 1
        
    elem[0] = bool_vec

for elem in clear_test:
    bool_vec = [0] * len(all_words)
    
    for word in elem[0]:
        bool_vec[all_words.index(word)] = 1
        
    elem[0] = bool_vec

In [26]:
main_clear_train[1]

array([list(['отпустить', 'коли', 'забирать', 'вещь', 'сажать', 'хренников', 'забирать', 'скорый', 'стать', 'плохо', 'камера', 'суд', 'завтра', 'новый', 'площадь', 'стр', 'написать', 'чириков', 'свой', 'блог', 'отношение', 'ечириков', 'составить', 'административный', 'протокол', 'статья', 'коап', 'город', 'москва', 'неповиновение', 'законный', 'требование', 'сотрудник', 'полиция']),
       -1], dtype=object)

In [27]:
sum(clear_train[1][0])

33

In [57]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

model = linear_model.LogisticRegression()
model = GridSearchCV(model, {"max_iter" : [100, 200, 300, 400]})

X = [bool_vec for bool_vec, rating in clear_train]
y = [rating for bool_vec, rating in clear_train]


X_test = [bool_vec for bool_vec, rating in clear_test]
y_test = [rating for bool_vec, rating in clear_test]

In [58]:
model.fit(X, y)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'max_iter': [100, 200, 300, 400]})

In [62]:
print(model.best_params_['max_iter'])

100


In [59]:
pred = model.predict(X_test)

In [61]:
from sklearn.metrics import f1_score, accuracy_score

f1_macro = f1_score(y_test, pred, average='macro')
f1_micro = f1_score(y_test, pred, average='micro')
f1_w = f1_score(y_test, pred, average='weighted')

acc = accuracy_score(y_test, pred)


print(f1_macro, f1_micro, f1_w, acc)

0.5683661045014432 0.5937021648808222 0.586470716322116 0.5937021648808222


***TF***

In [32]:
tf_test = main_clear_test.copy()
tf_train = main_clear_train.copy()

In [33]:
#рассчет TF

for elem in tf_train:
    tf_vec = [0] * len(all_words)
    
    for word in elem[0]:
        tf_vec[all_words.index(word)] += 1
        
    elem[0] = tf_vec

for elem in tf_test:
    tf_vec = [0] * len(all_words)
    
    for word in elem[0]:
        tf_vec[all_words.index(word)] += 1
        
    elem[0] = tf_vec

In [34]:
model = linear_model.LogisticRegression(max_iter = 350)

X = [tf_vec for tf_vec, rating in tf_train]
y = [rating for tf_vec, rating in tf_train]


X_test = [tf_vec for tf_vec, rating in tf_test]
y_test = [rating for tf_vec, rating in tf_test]

In [35]:
model.fit(X, y)

LogisticRegression(max_iter=350)

In [36]:
pred = model.predict(X_test)

In [37]:
f1_macro = f1_score(y_test, pred, average='macro')
f1_micro = f1_score(y_test, pred, average='micro')
f1_w = f1_score(y_test, pred, average='weighted')

acc = accuracy_score(y_test, pred)


print(f1_macro, f1_micro, f1_w, acc)

0.5675216357691751 0.5934834900502952 0.5859657167459726 0.5934834900502952


***TF-IDF***

In [38]:
idf_test = main_clear_test.copy()
idf_train = main_clear_train.copy()

In [47]:
#рассчет TF-IDF

#df - кол-во документов, содержащих терм

df = [0] * len(all_words)

for elem in idf_train:
    for i in range(len(all_words)):
        if all_words[i] in elem[0]:
            df[i] += 1
            
for elem in idf_test:
    for i in range(len(all_words)):
        if all_words[i] in elem[0]:
            df[i] += 1
            
df = np.array(df)
            
idf = np.log10(len(idf_train) / df)

In [49]:
#рассчет TF

for elem in idf_train:
    idf_vec = [0] * len(all_words)
    
    for word in elem[0]:
        idf_vec[all_words.index(word)] += 1
    idf_vec = np.multiply(idf_vec, idf)
        
    elem[0] = idf_vec

for elem in idf_test:
    idf_vec = [0] * len(all_words)
    
    for word in elem[0]:
        idf_vec[all_words.index(word)] += 1
    idf_vec = np.multiply(idf_vec, idf)
        
    elem[0] = idf_vec

In [50]:
model = linear_model.LogisticRegression(max_iter = 350)

X = [idf_vec for idf_vec, rating in idf_train]
y = [rating for idf_vec, rating in idf_train]


X_test = [idf_vec for idf_vec, rating in idf_test]
y_test = [rating for idf_vec, rating in idf_test]

In [51]:
model.fit(X, y)

LogisticRegression(max_iter=350)

In [52]:
pred = model.predict(X_test)

In [53]:
f1_macro = f1_score(y_test, pred, average='macro')
f1_micro = f1_score(y_test, pred, average='micro')
f1_w = f1_score(y_test, pred, average='weighted')

acc = accuracy_score(y_test, pred)


print(f1_macro, f1_micro, f1_w, acc)

0.566732683814627 0.5930461403892412 0.5848899693832185 0.5930461403892412


***Отчет***


В данном задании было реализовано обучение модели с использованием булевых векторов, TF и TF-IDF. Во всех трех случаях моделью выбрана LogisticRegression. Дополнительно тестировался SGDClassifier, но он показал результат заметно хуже, в связи с чем был удален из кода.


Accuracy на всех трех вариантах представления данных держится около 0.59, меры F1_macro, F1_micro и F1_weighted соответственно на уровне 0.56, 0.59 и 0.58.


Из предостваленного XML файла в данные для обучения был выделен только текст и его рейтинг ("-", "0", "+", остальные игнорировались вместе с текстом). Из полученных данных были удалены латинские символы, цифры, знаки препинания и другие знаки (marks). После разбиения на слова была проведена нормализация, что существенно уменьшило размер вектора уникальных слов: с 44 тысяч до 22 тысяч слов. Текст рассматривался как мешок слов (bag-of-words), без учета контекста.


Был протестирован метод Grid Search на булевых векторах, который почему-то выполняется крайне долго. При ручном запуске сети с параметром max_iter = 350 сеть обучалась около минуты, обучение с Grid Search на 100, 200, 300 и 400 время обучения достигало 15 минут, причем оптимальным параметром оказывался max_iter = 100, в связи с чем в других представлениях данных Grid Search не применялся. 